## This notebook generates a bench of plot to show different parameters of task performance

### This notebook is used after data processing to plot the data. Do not play with it lightly inside the master folder (load_preprocess_mouse)

### This notebook is relying on the load_Rat_documentation folder which is doing preprocessing the files generated durting the experiments and generate for each experiment the class Data

#### 1. Only modifiy if you are sure of what you are doing and that you are solving a bug
#### 2. If you do modify you MUST commit this modification using bitbucket
#### 3. If you want to play whis notebook (to understand it better) copy it on a toy folder distinct from the master folder
#### 4. If you want to modify this code (fix bug, improve, add attributes ...) it is recommanded  to first duplicate in a draft folder. Try to keep track of your change.
#### 5. When you are ready to commit : # clear all output, clean everything between hashtag 


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import pandas as pd
import scipy
import math
from scipy.interpolate import interp1d
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import collections  as mc
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1.inset_locator as inset
%matplotlib inline

import glob
import os
import platform
import pickle
import datetime
from scipy import stats

## The lines below allow to run required notebook from the master folder
if "__file__" not in dir():
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommunNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    os.chdir(CommunNoteBookesPath)
    
    %run Animal_Tags.ipynb
    %run UtilityTools.ipynb
    %run loadRat_documentation.ipynb
    #%run loadRawSpike_documentation.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    

if "__file__" not in dir():

    if platform.system()=='Linux':
        root="/data"
    else:
        root="/Users/davidrobbe/Documents/Data/"

    print("The path to data is %s"%root)


    
# INFO: all the default parameters for preprocessing (not used in the notebook, just for info)
defaultParam={
    "binSize":0.25,
    "trialOffset":20., #max end of trial, in seconds (position will be cutted)
    "sigmaSmoothPosition":0.1,  #smooth the position
    #"sigmaSmoothPosition":0.33 for pavel dataType
    "sigmaSmoothSpeed":0.3, #smooth the speed
    "positionDiffRange": [2.,5.], #min and max differences allowed between two consecutive positions
                                  #min to correct start, max to correct jumps
    "pawFrequencyRange":[2.,10.],
    "startAnalysisParams":[10,0.2,0.5],
    "cameraToTreadmillDelay":2., #seconds, usual time between camera start and treadmill start
    "nbJumpMax" : 100., #if jumps>nbJumpMax, trial is badly tracked
    
    #parameter to detect end of trial (first position minima)
    "endTrial_backPos":55,  # minima is after the animal went once to the back (after first time position>backPos)
    "endTrial_frontPos":30, # minima's position is in front of treadmill (position[end]<frontPos)
    "endTrial_minTimeSec":4, # minima is after minTimeSec seconds (time[end]>minTimeSec)
    }

### Load Data
Load the preprocess data (corrected and binned position, speed, median position, ...)
 DEFAULT: load pickle if they exist, or create them
data=Data(root,animal,experiment,param=param)

 OPTION 1: do not save any new pickle file
data=Data(root,animal,experiment,param,saveAsPickle=False)

 OPTION 2: redo the preprocessing with param, even if the pickle already exist
 (to be sure everything is preprocess with the same parameters)

In [ ]:
#run only if inside this notebook (do not execute if "%run this_notebook")
if "__file__" not in dir():    
    
    experiment="Rat147_2017_09_05_11_52"
    
    animal= experiment[:6]
    
    param={
        "goalTime":7,
        "treadmillRange":[0,90],
        "maxTrialDuration":15,
        "interTrialDuration":10,
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "binSize":0.25,
    }  
    data=Data(root,animal,experiment,param,redoPreprocess=True,PrintWarning=True)
    
    
#Description of the session parameters
    #if data.hasBehavior:
     #   data.describe()
    

In [ ]:
#trial=0
#print(data.entranceTime[trial])
#plt.plot(data.timeTreadmill[trial],data.position[trial])
#plt.axvline(data.entranceTime[trial],color='r')
#plt.show()

### General plot behavior

In [ ]:
def plot_session_behavior(data):
    try :
        TagRat = data.tag
        
            
    except AttributeError:
        TagRat=""
    
    
    
    
    fig=plt.figure(figsize=(15,10))
    
    
    
    fig.add_subplot(331)
    try:
        plot_entrance_time(data)
    except Exception as e:
        plt.title(str(e))


    fig.add_subplot(332)
    try:
        plot_positions(data)
        #plt.xlim(0,12)
    except Exception as e:
        plt.title(str(e))

    
    fig.add_subplot(333)
    try:
        plot_position_align_end_StereotypedTrials(data,onlyGood=False)
    except Exception as e:
        plt.title(str(e))
   
    fig.add_subplot(337)
    try:
        plot_correlation_position(data,onlyGood=False)
    except Exception as e:
        plt.title(str(e))


    fig.add_subplot(338)
    try:
        plot_rmse(data,alignedOnBegining=False)
        plt.gca().set_title(plt.gca().get_title()+'\naligned on trajectory peak')
    except Exception as e:
        plt.title(str(e))

    fig.add_subplot(334)
    try:
        plot_trajectory_PDF(data)
    except Exception as e:
        plt.title(str(e))

    
    fig.add_subplot(335)
    try:
        hist=plot_kinematic_histo(data,kinematic="speedSmoothBin",title="Speed",xlablel="speed(cm/s)",color='b',alpha=0.8)
        hist=plot_kinematic_histo(data,kinematic="speedSmoothBin",title="Speed",xlablel="speed(cm/s)",color='r',alpha=0.5,onlyGood=True)
        plt.title("Speed (blue:all trials, red: only good)")
    except Exception as e:
        plt.title(str(e))
    
    ax=plt.subplot2grid(shape=(3, 3), loc=(1,2), colspan=1, rowspan=2)
    try:
        plot_lick_raster(data,axes=ax,plot_inset=False)
    except Exception as e:
        plt.title(repr(e))
    
        
    title=TagRat+ ", "+data.experiment+", day %s, %s trials"%(data.daySinceStart,data.nTrial)
    fig.suptitle(title,fontsize=16,y=1.03)
    
    fig.subplots_adjust(hspace=.3)
    try:
        fig.tight_layout()
    except:pass
#----------------------------------------------------------------------------------------------------------------------
if "__file__" not in dir():
    session='Rat220_2018_04_24_10_49'
    data=Data(root,session[:6],session,redoPreprocess=False)
    plot_session_behavior(data)  
    
    #path=os.path.join(root,"Analysis")
    #pathFig=os.path.join(path,"Rat051_2016_05_06_16_04.png")
    #plt.savefig(pathFig)
    

In [ ]:
# # Looking at different time axes attributes
# if "__file__" not in dir():
#     trial=33
#     plt.plot(data.rawTime[trial],data.rawPosition[trial],'b-')
#     plt.plot(data.timeTreadmill[trial],data.position[trial],"g-")
#     plt.plot(data.timeBin,data.positionBin[trial],'ro')#centered
#     plt.plot(data.lickTime[trial],np.zeros(len(data.lickTime[trial])),'mo')
#     plt.ylabel("position(cm)")
#     plt.xlabel("time(s)")
#     plt.ylim(-5, 90)
#     print(data.maxTrialDuration)

## Plot Behavioral data : TreadmillOn
###  Different position representations (trajectories of the animal)

In [ ]:
if "__file__" not in dir():
    trial=0
    plt.figure(figsize=(12,8))
    plt.subplot(325)
    for trial in data.trials:
        plt.plot(data.timeBin,data.positionBin[trial],'k-',alpha=0.2)
    plt.plot(data.timeBin,data.medianPosition,'r');
    plt.title("Median of corrected position");

    plt.subplot(322)
    plt.plot(data.timeBin,data.positionBin[trial],'bo-')
    plt.title("position smooth and binned for trial %s"%trial+'\n cut at the end of trial')
    plt.legend();
    
    plt.subplot(324)
    for trial in data.positionAlignEnd:
        plt.plot(data.timeAlignEnd,data.positionAlignEnd[trial],'k-',alpha=0.2)
    plt.title("Position align on end (%s)"%len(data.positionAlignEnd))
    
    plt.subplot(323)
    for trial in data.trials:
        plt.plot(data.timeBin,data.positionBin[trial],'k-',alpha=0.2)
    plt.title("Position align on treadmill start")
    
    plt.subplot(326)
    for trial in data.positionAlignEnd:
        plt.plot(data.timeAlignEnd,data.positionAlignEnd[trial],'k-',alpha=0.2)

    plt.plot(data.timeAlignEnd,data.medianPositionAlignEnd,'r')
    plt.title("median align on end")
    
    plt.subplot(321)
    trial=40
    plt.plot(data.timeTreadmill[trial],data.position[trial])
    plt.title("position(smooth, no bin), aligned on treadmill start")

### Positions, percentile, correlation

  - `onlyGood (False/True)`: plot/compute only on good trials
  - `raw (False/True)`: use raw positions. If False, use the corrected smoothed position.

In [ ]:
def get_positions_array_beginning(data,onlyGood=False,raw=False):
    '''
    Return all the position in one array, between startFrame (treadmill start) and stopFrame (treadmill stop)
    '''
    if raw:
        posDict=data.rawPosition
    else:
        posDict=data.position
    #the mean goal time, converted in number of frames    
    
    #put all positions in a 2D array
    allTraj=[]
    pos={}
    for trial in posDict:
        if onlyGood and (trial not in data.goodTrials):
            continue
        start = int(data.startFrame[trial])
        stop = int(data.stopFrame[trial])#endIndex#int(data.startFrame[trial]+size)
        pos[trial]=posDict[trial][start:stop]
    if len(pos)>0:
        size=np.max([len(pos[trial]) for trial in pos])

    for trial in pos:
        #pad with nan at the end if too short (entrance time < mean goal time)
        if len(pos[trial])<(size):
            pos[trial]=np.append(pos[trial],[np.nan] * (size-len(pos[trial])))
        allTraj.append(pos[trial])
    
    allTraj=np.asarray(allTraj)
    return allTraj

In [ ]:
if "__file__" not in dir():

    session='Rat154_2017_09_25_13_58'
    data=Data(root,session[:6],session,redoPreprocess=False)
    plot_run_distance(data,onlyGood=False,raw=False)

In [ ]:
def find_persistent_movement_epochs(pos: np.array, TH: int, treadmillStep: float =0):
    """
    this function finds epochs in the position of a single trial, pos, where the animal has moved
    for a duration of at least, TH*cameraSamplingRate
    pos: position as a np.array
    TH: an integer like: TH=int(0.2*data.cameraSamplingRate)
    treadmillStep: a float number to correct for treadmill movement, like:
        treadmillStep=np.expand_dims(data.treadmillSpeed[trial]/data.cameraSamplingRate,1)
    """
    
    dp=np.append(np.diff(pos)-treadmillStep,0)
    dp=dp>=0
    dp=dp*1
    dpcross=np.absolute( np.append(np.diff(dp),0)-np.append(0,np.diff(dp)) )
    #any 1 in dpcross mean a zero-crossing in diff(pos)
    crossindex=np.where(dpcross)[0]
    movementepoch=np.where(np.diff(crossindex)>TH)[0]

    posMoveIndex=np.empty(shape=(len(movementepoch),2))

    for i,index in enumerate(movementepoch):
        posMoveIndex[i,:]=(crossindex[index],crossindex[index+1])
        
    return posMoveIndex

def run_distance(data,trial=None):
    if trial is None:
        trial=list(data.position.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
        
    dis=[0]
    for i in trial:
        et=(data.entranceTime[i]+data.cameraToTreadmillDelay)*data.cameraSamplingRate
        allTraj=data.position[i]
    #     allTraj=scipy.ndimage.filters.gaussian_filter1d(allTraj, sigma=data.cameraSamplingRate/5, truncate=3.0)
        if np.isnan(et):
            th=(data.maxTrialDuration[i]+data.cameraToTreadmillDelay)*data.cameraSamplingRate
            th=int(th)
        else:
            th=int(et)
        allTraj[th-1:]=np.nan
        treadmillStep=np.expand_dims(data.treadmillSpeed[i]/data.cameraSamplingRate,1)
        posMoveIndex=find_persistent_movement_epochs(allTraj,
                                                     TH=int(0.6*data.cameraSamplingRate),
                                                     treadmillStep=treadmillStep)

        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)-treadmillStep))
        dis.append(0)

    return np.array(dis[:-1])

def intertrial_displacement(data,trial=None):
    if trial is None:
        trial=list(data.position.keys())
    elif not hasattr(trial,'__iter__'):
        trial=[trial]
    
    dis=[0]
    for i in trial:
        if data.dataType != 'ViewPoint':
            allTraj= data.position[i][int((-data.interTrialDuration[i]+1)*data.cameraSamplingRate):]
            TH=int(0.6*data.cameraSamplingRate)
        else:
            allTraj=data.rawIntertrialPosition[i]
            TH=int(0.6*data.cameraSamplingRateInterTrial)

        posMoveIndex=find_persistent_movement_epochs(allTraj,
                                                     TH=TH,
                                                     )

        for i,j in posMoveIndex:
            epochPos=allTraj[int(i):int(j)]
            dis[-1]+=np.nansum(np.absolute(np.diff(epochPos)))
        dis.append(0)

    return np.array(dis[:-1])

def plot_run_distance(data):
    dis=run_distance(data)
    
    plt.plot(dis,"-")
    plt.ylabel("run distance (cm)")
    plt.xlabel("Trial")
    title="Rat run distance"
    plt.title(title)

    return dis
#----------------------------------------------------------------------------------------------------------------------
def plot_positions(data,onlyGood=False,raw=False,vSpan=True,**kargs):
    '''
    Plot the positions (green=good trial, red=other), aligned on camera start
    Plot the percentile (25%, 50%, 75%) on the beginning (trial start to trial stop (treadmill stop))
    '''
    try:
        ax=kargs['axis']
        kargs.pop('axis', None)
    except:
        ax=plt.gca()
    
    if raw:
        posDict=data.rawPosition
    else:
        posDict=data.position
    time=data.rawTime #align on camera
    
    for trial in posDict:
        kargs['color']="green"
        if trial not in data.goodTrials:
            if onlyGood:
                continue
            kargs['color']="red"

        ax.plot(time[trial][:data.stopFrame[trial]],posDict[trial][:data.stopFrame[trial]],**kargs)
        
    #Get the positions between trial start and mean goal time
    allTraj=get_positions_array_beginning(data,onlyGood=onlyGood,raw=raw)
    #Check that the array is not empty
    if allTraj.shape[0]==0:
        if onlyGood:
            ax.set_title("0 Good trials")
        ax.set_title("No positions")
        return False

    #plot the percentiles
    trajP=np.nanpercentile(allTraj,[25, 50, 75],axis=0)
    time=(np.arange(allTraj.shape[1])/data.cameraSamplingRate)+data.cameraToTreadmillDelay
    #plt.plot(time,trajP[0],"b--",lw=2)
    ax.plot(time,trajP[1],"b-",lw=2)
    #plt.plot(time,trajP[2],"b--",lw=2)
    #title, labels, grid
    ax.set_ylabel("X Position (cm)")
    ax.set_xlabel("Time (s) relative to camera start")
    #plt.xlim([0,max(data.entranceTime)])
    ax.set_xlim([0,16])
    ax.grid()
    if vSpan:
        ax.axvspan(data.cameraToTreadmillDelay,data.cameraToTreadmillDelay+np.nanmedian(data.goalTime),alpha=0.2,color="grey")
    title="Rat trajectories"
    ax.set_title(title)
    
def get_positions_array_peaks_aligned(data,onlyGood=False,raw=False):
    '''
    Returns array of position, align on detected peak,
    If one position is too short (detected end is before minTime), 
      the position is pad at the beginning and end with nan values
    '''
    if not data.trials:
        return np.array([])
    if raw:
        posDict=data.rawPosition
    else:
        posDict=data.position
        
    #number of frames to keep 
    cs=data.cameraSamplingRate
    size=int(np.nanmax(data.maxTrialDuration)*cs)+1 #int(abs(minTime*cs))
    #put all positions in a 2D array
    allTraj=np.ones(shape=(data.trials[-1]+1,size*2))*np.nan  #matrix of trial position*time
    for trial in posDict:
        if onlyGood and (trial not in data.goodTrials):
            continue
        
        #index where the trajectory begins and ends
        endIndex  =int(data.stopFrame[trial])
        startIndex=int(data.startFrame[trial])
        pos=posDict[trial][startIndex:endIndex]
        peakIndex=np.argmax(pos)
        indexOffset=size-peakIndex
        allTraj[trial,indexOffset:indexOffset+len(pos)]=pos
    return allTraj

#----------------------------------------------------------------------------------------------------------------------
def plot_correlation_position(data,onlyGood=False,raw=False):
    '''
    Compute the correlation of the position between trial start and trial stop (treadmill stop)
    Plot the matrix of correlation coefficients
    Returns the median of coefficients
    '''
    allTraj=get_positions_array_beginning(data,onlyGood=onlyGood,raw=raw)

    df = pd.DataFrame(allTraj.transpose())

    if allTraj.shape[0]<3:
        title="Not enough trials"
        med=np.nan
    else:
        corMatrix=np.array(df.corr())
        
        pp=plt.pcolor(corMatrix,vmin=-1,vmax=1,cmap="Reds")
        plt.colorbar(pp)
        plt.xlim([0,corMatrix.shape[0]])
        plt.ylim([0,corMatrix.shape[1]])
        #median of upper triangle of matrix. carefull not to take the 
        coef=corMatrix[np.tril_indices(corMatrix.shape[0],-1)]
        #print(len(coef))
        med=np.nanmedian(coef)
        maxSecond=allTraj.shape[1]/float(data.cameraSamplingRate)
        #title of the plot
        title=""
        if onlyGood:
            title="Good Trials"
        else:
            title="All Trials"
        
        title+=', trajectory median  r= %.2f'%med     
    plt.title(title)
    #print(med)
    return med

def plot_rmse(data,alignedOnBegining=True,onlyGood=False,raw=False):
    '''
    Compute the rmse of the position between trial start and trial stop (treadmill stop)
    plots and returns the RMSE matrix
    '''
    if alignedOnBegining:
        allTraj=get_positions_array_beginning(data,onlyGood=onlyGood,raw=raw)
    else:
        allTraj=get_positions_array_peaks_aligned(data,onlyGood=onlyGood,raw=raw)
    
    allTraj=allTraj[:,int(data.cameraToTreadmillDelay*data.cameraSamplingRate):]
    
    NbTrial=allTraj.shape[0]
    
    if NbTrial<3:
        title="Not enough trials"
        med=np.nan
        return med

    rmse=np.ones((NbTrial,NbTrial))*(-1)

    for i in range(NbTrial):
        rmse[i,i]=0
        for j in range(i+1,NbTrial):
            
            a=allTraj[i,:]-allTraj[j,:]
            rmse[i,j]=np.sqrt(
                             np.nansum(a**2) / np.sum( np.logical_not(np.isnan(a)) )
                             )
    
    RMSEmatrix=np.triu(rmse,k=0)+np.triu(rmse,k=0).T #symetrical
    
    #median of upper triangle of matrix
    coef=RMSEmatrix[np.tril_indices(RMSEmatrix.shape[0],-1)]
    #print(len(coef))
    m=np.nanmedian(coef)

    RMSEmatrix/=data.treadmillRange[1]
    pp=plt.pcolor(RMSEmatrix,vmin=0,vmax=1,cmap="Reds")
    plt.colorbar(pp)
    plt.xlim([0,RMSEmatrix.shape[0]])
    plt.ylim([0,RMSEmatrix.shape[1]])
    
    
    #title of the plot
    title="RMSE of "
    if onlyGood:
        title+="Good Trials"
    else:
        title+="All Trials"

    title+=', mean= %.2f'%m    
    plt.title(title)

    
    return m,RMSEmatrix

def plot_trajectory_PDF(data,TimeRes=.5,PosRes=5,onlyGood=False):
    """
    calculates and plots the joint PDF of trajectories.
    time resolution in seconds
    Position resolution in cm
    """
    
    if not data.trials:
        ax=plt.gca();
        return np.nan
    allTraj=get_positions_array_beginning(data,onlyGood).T
    trialDuration=np.nanmax(data.maxTrialDuration)

    posSize =len(np.arange(data.treadmillRange[0],data.treadmillRange[1],PosRes))
    timeSize=int((allTraj.shape[0]/data.cameraSamplingRate)//TimeRes)#len(np.arange(0,trialDuration,TimeRes))
    trajDis=np.zeros([timeSize+1,posSize])
    
    allTraj=allTraj//PosRes

    for t in range(allTraj.shape[0]):
        timeIndex=int((t/data.cameraSamplingRate)//TimeRes)
        trajDis[timeIndex,:]=[np.sum(allTraj[t,:]==x) for x in range(posSize)]

    trajDis=scipy.ndimage.filters.gaussian_filter(trajDis, sigma=[1,1],
                                                  order=0, mode='nearest', truncate=3)
    #normalizing as a PDF
    trajDis/=np.sum(trajDis)
    H=twoD_entropy(trajDis)
    
    ax=plt.gca();
    ax.pcolor(trajDis.T, cmap=cm.Reds);
    
    ax.set_xticks     (np.linspace(0,timeSize,5));
    ax.set_xticklabels(np.linspace(0,trialDuration,5));
    ax.set_yticks     (np.linspace(0,posSize,10));
    ax.set_yticklabels(np.linspace(data.treadmillRange[0],data.treadmillRange[1],10));
    ax.set_title("Trajectory PDF: H="+"%.2f"%H+' bits')
    ax.set_xlabel("Time(s)")
    ax.set_ylabel("Position(cm)")
    
    return trajDis,H

def twoD_entropy(trajDist):
    H=0
    for i in range(trajDist.shape[0]):
        for j in range(trajDist.shape[1]):
            try:
                H+=trajDist[i,j]*math.log(float(trajDist[i,j]),2)
            except :
                pass
    H=-H
    return H

In [ ]:
#----------------------------------------------------------------------------------------------------------------------
if "__file__" not in dir():
    session="Rat160_2017_08_21_15_36"
    data=Data(root,animal=session[:6],experiment=session,param=defaultParam,redoPreprocess=False,PrintWarning=True)
    plt.figure(figsize=(10,5))
#     plt.subplot(221)
    plot_positions(data)
#     plot_trajectory_PDF(data)
#     plt.subplot(222)
#     plot_correlation_position(data,onlyGood=True,raw=False)
#     plt.subplot(121)
#     allTraj=get_positions_array_peaks_aligned(data,onlyGood=False,raw=False)
#     plt.plot(allTraj.T)
#     plt.subplot(122)
#     allTraj=get_positions_array_beginning(data,onlyGood=False,raw=False)
#     plt.plot(allTraj.T)
#     plot_rmse(data,alignedOnBegining=False,onlyGood=False,raw=False)

In [ ]:
def plot_mediantrajectory(data,onlyGood=False,raw=False):
    '''
    Plot the positions (green=good trial, red=other), aligned on camera start
    Plot the percentile (25%, 50%, 75%) on the beginning (trial start to trial stop (treadmill stop))
    '''
    if raw:
        posDict=data.rawPosition
    else:
        posDict=data.position
    time=data.rawTime #align on camera   
    #Get the positions between trial start and mean goal time
    allTraj=get_positions_array_beginning(data,onlyGood=onlyGood,raw=raw)
    #Check that the array is not empty
    if allTraj.shape[0]==0:
        if onlyGood:
            plt.title("0 Good trials")
        plt.title("No positions")
        return False

    #plot the percentiles
    trajP=np.nanpercentile(allTraj,[25, 50, 75],axis=0)
    corr=np.zeros( allTraj.shape[0])
    median=trajP[1]
    for trial in range(allTraj.shape[0]):
        traj=allTraj[trial,:]
        corr[trial]=scipy.stats.spearmanr(median[~np.isnan(traj)],traj[~np.isnan(traj)])[0]
    time=(np.arange(allTraj.shape[1])/data.cameraSamplingRate)+data.cameraToTreadmillDelay
    plt.plot(time,trajP[0],"b--",lw=2)
    plt.plot(time,trajP[1],"b-",lw=2)
    plt.plot(time,trajP[2],"b--",lw=2)
    
    #Get the positions between trial start and mean goal time for good trial only
    allTraj=get_positions_array_beginning(data,onlyGood=True,raw=raw)
    #Check that the array is not empty
    if allTraj.shape[0]==0:
        if onlyGood:
            plt.title("0 Good trials")
        plt.title("No positions")
        return False

    #plot the percentiles
    trajP=np.nanpercentile(allTraj,[25, 50, 75],axis=0)
    corr=np.zeros( allTraj.shape[0])
    median=trajP[1]
    for trial in range(allTraj.shape[0]):
        traj=allTraj[trial,:]
        corr[trial]=scipy.stats.spearmanr(median[~np.isnan(traj)],traj[~np.isnan(traj)])[0]
    time=(np.arange(allTraj.shape[1])/data.cameraSamplingRate)+data.cameraToTreadmillDelay
    plt.plot(time,trajP[0],"r--",lw=2)
    plt.plot(time,trajP[1],"r-",lw=2)
    plt.plot(time,trajP[2],"r--",lw=2)
    
    #title, labels, grid
    plt.ylabel("X Position (cm)")
    plt.xlabel("Time (s) relative to camera start")
    #plt.xlim([0,max(data.entranceTime)])
    plt.xlim([0,16])
    plt.grid()
    title=""
    if raw:
        title="Median Raw "
    if not raw:
            title+="Median Smooth "
    if onlyGood:
        title+="trajectory %s good trials"%(len(data.goodTrials))
    else:
        title+="trajectory, %s good / %s trials"%(len(data.goodTrials),len(data.trials))
    title+=' , \n avg correlation of trajectories with the median trajectory '+np.str(np.round(np.mean(corr),2))
    title+="\n good trials in red"
    plt.title(title)
    plt.ylim(0,90)
    return trajP
if "__file__" not in dir():
    plot_mediantrajectory(data,onlyGood=False)
    #plot_mediantrajectory(data,onlyGood=True)

### Position align on end and correlation

The end of each trial trajectory is detected during preprocessing with the parameters:

 - "endTrial_backPos":55,  minima is after the animal went once to the back (after first time position>backPos)
 - "endTrial_frontPos":30,  minima's position is in front of treadmill (position[end]\<frontPos)
 - "endTrial_minTimeSec":4, minima is after minTimeSec seconds (time[end]>minTimeSec)

The detection is done on the corrected smoothed position.  
`data.indexEndTrial` contains the detected index for each trial. If the end could not be detected, `indexEndTrial[trial]=None`.  
For more details see Preprocess_treadmillOn_detailed_documentation.ipnb

  - `minTime (int)`: time in seconds to consider, relative to detected end  
    `minTime= -5` means "keep 5 seconds before the end", for each trial
  - `onlyGood (True/False)`: whether to keep only good trials
  - `raw (True/False)`: whether to use raw positions

In [ ]:
def get_positions_array_end_StereotypedTrials(data,onlyGood=False,raw=False):
    '''
    Returns array of position, align on detected end, between minTime and zero (0=end)
    The dectect end can be None, if so the trial is skipped
    If one position is too short (detected end is before minTime), 
      the position is pad at the beginning with nan values
    '''
    if raw:
        posDict=data.rawPosition
    else:
        posDict=data.position
    #number of frames to keep 
    cs=data.cameraSamplingRate
    size=np.max(data.maxTrialDuration)*cs#int(abs(minTime*cs))
    #put all positions in a 2D array
    allTraj=[]
    for trial in posDict:
        if onlyGood and (trial not in data.goodTrials):
            continue
        #index where the steretyped trajectory ends
        endIndex=data.indexEndTrial[trial]
        if isNone(endIndex):
            #no end was detected, skipp trial
            continue 
        #Position is cut between (end -minTime) and end.
        #At minima, the cut should begin at startFrame
        startIndex=data.startFrame[trial]
        pos=posDict[trial][startIndex:int(endIndex)]
       
        if len(pos)<(size):
            #pos=np.append([pos[0]] * (size-len(pos)),pos) #if too short, pad the start with a plateau (Nan would cause trouble to compute speed later)
            pos=np.append(np.nan * int(size-len(pos)),pos)
        allTraj.append(pos)
    allTraj=np.asarray(allTraj)
    return allTraj
#----------------------------------------------------------------------------------------------------------------------
def plot_position_align_end_StereotypedTrials(data,xUpLimit=5,onlyGood=False):
    '''
    [minTime to 0] is the time range to measure the correlation, with 0=detected end
    xUpLimit is the x axis max limit
    onlyGood is whether to consider only good trials
    nb: data.timeEndTrial is computed with data.position
    '''
    cs=data.cameraSamplingRate
    for trial in data.position:
        end=data.timeEndTrial[trial]
        stop=data.stopFrame[trial]
        start=data.startFrame[trial]#+data.cameraToTreadmillDelay*data.cameraSamplingRate
        indexStart=np.argmax(data.position[trial])
        indexEnd=data.indexEndTrial[trial]
        if isNone(end):
            continue
        color="green"
        if trial not in data.goodTrials:
            if onlyGood:
                continue
            color="red"
        plt.plot(data.timeTreadmill[trial][start:stop]-end , data.position[trial][start:stop],color=color)
        plt.plot(data.timeTreadmill[trial][indexStart:indexEnd]-end,data.position[trial][indexStart:indexEnd],color="b",lw=2)
    plt.xlim([-15,xUpLimit])
    plt.ylabel("Position (cm/s)")
    plt.xlabel("Time relative to detected end (s)")
    plt.axvline(0,color="b",ls="--")
    plt.axvline(-np.mean(data.goalTime),color="k",ls="--")
    w="correct " if onlyGood else ""
    
    allTraj=get_positions_array_end_StereotypedTrials(data,onlyGood=onlyGood,raw=False)
    df = pd.DataFrame(allTraj.transpose())
    if allTraj.shape[0]<3:
        title="Not enough trials"
        med=np.nan
    else:
        corMatrix=np.array(df.corr())
        coeff=corMatrix[np.triu_indices(corMatrix.shape[0],1)]
        med=np.nanmedian(coeff)
    plt.title("Aligned to minima of stereotyped trajectory")    
    plt.axvspan(-np.max(data.maxTrialDuration),0,alpha=0.2,color="grey")
    return med

#%%%%%%%%%%%%%%%%%%%%%%%%%MOSTAFA%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# THIS FUNCTION IS DEPRECATED! AVOID USING
def plot_forward_running_speed(data,minXBack=55,onlyGood=False):
    '''
    This function detects the running forward portion of the 
    trials (where x > minXBack) and returns the forward speed
    
    '''
    forwardSpeed=[]
    cs=data.cameraSamplingRate
    for trial in data.position:
        end=data.timeEndTrial[trial]
        indexStart=np.argmax(data.position[trial])
        indexEnd=data.indexEndTrial[trial]
        XData=data.position[trial][indexStart:indexEnd]
        if isNone(end) or data.position[trial][indexStart]<minXBack or len(XData)<=2:
            continue
        
        color="green"
        if trial not in data.goodTrials:
            if onlyGood:
                continue
            color="red"
        
        forwardSpeed.append(abs((XData[0]-XData[-1])/(indexStart-indexEnd))*cs+data.treadmillSpeed[trial])
        plt.plot(data.timeTreadmill[trial][indexStart:indexEnd]-end,XData,color=color,lw=2)

    plt.xlim([-15,0])
    plt.ylabel("Position (cm/s)")
    plt.xlabel("Time relative to detected end (s)")
#     plt.axvline(0,color="b",ls="--")
    plt.axvline(-np.mean(data.goalTime),color="k",ls="--")
    plt.title("mean forward acceleration speed {}".format(np.mean(forwardSpeed)))    
#     plt.axvspan(-np.max(data.maxTrialDuration),0,alpha=0.2,color="grey")   
    


class forwardRunningSpeed:
    '''
    This class detects the running forward portion of the 
    trials and returns the forward speed
    
    '''
    def __init__(self,data,minXBack=60,minXFront=40):
        assert isinstance(data,Data), 'input data is not an instance of Data class'
        
        data.position_correction()
        self.cs          =data.cameraSamplingRate
        self.posDict     =data.position
        self.goodTrials  =data.goodTrials
        self.treadmillSpd=data.treadmillSpeed
        self.et          =data.entranceTime
        self.timeEndTrial=detect_trial_end(data)
        self.cameraDelay =data.cameraToTreadmillDelay
        self.xBack       =minXBack
        self.xFront      =minXFront
        self.session     =data.sessionPath
        self.forwardSpeed=None
    
    def compute(self):
        forwardSpeed={}
        for trial,pos in self.posDict.items():
            back_P = (pos >self.xBack).astype(int)
            _backCross=back_P-np.concatenate((np.array([False]),back_P[:-1]))
            backCrosses=np.where(_backCross==-1)[0]
            if len(backCrosses)==0:
                continue
            else:
                backCross=backCrosses[0]

            front_P= (pos >self.xFront).astype(int)
            _frontCross=front_P-np.concatenate((np.array([False]),front_P[:-1]))
            frontCrosses=np.where(_frontCross==-1)[0]
            #detect first front cross after a back cross
            frontCross=frontCrosses[frontCrosses>backCross]
            if len(frontCross)==0:
                continue
            else:
                frontCross=frontCross[0]
                            
            if frontCross/self.cs > self.timeEndTrial[trial]+self.cameraDelay:
                # treadmill had already stopped, so discard
                continue
            
            forwardSpeed[trial]=(
                                 abs(
                                     (pos[backCross]-pos[frontCross])/(frontCross-backCross)
                                    )*self.cs + self.treadmillSpd[trial]
                                 )
        
        self.forwardSpeed=forwardSpeed

        return self.forwardSpeed
                
    def conditions(self, goodOnly=False, last_n=False, first_n=False, sharpOnly=False):
        """
        Turn any of the input arguments, in order
        to apply that condition. Applying conditions must precede 
        running the compute() method.
        """
        assert self.forwardSpeed is None, "conditions() must be run before compute()"
        
        if goodOnly:
            self._good_only()
        if last_n:
            self._last_n(last_n)
        if first_n:
            self._first_n(first_n)
        if sharpOnly:
            self._sharp_only(sharpOnly)
        
    def _good_only(self):
        self.posDict={trial:pos 
                      for trial,pos in self.posDict.items() 
                      if trial in self.goodTrials}
        
    def _last_n(self,last_n):
        if not last_n or not isinstance(last_n,int):
            return
        nIndex=len(self.posDict)-last_n
        self.posDict={trial:pos 
                      for trial,pos in self.posDict.items() 
                      if trial >= nIndex}

    def _first_n(self,first_n):
        if not first_n or not isinstance(first_n,int):
            return
        nIndex=first_n
        self.posDict={trial:pos 
                      for trial,pos in self.posDict.items() 
                      if trial <= nIndex}
    
    def _sharp_only(self,sharpOnly: tuple):
        """
        sharpOnly: a tuple (a,b) where only trials with  a<ET<b are considered
        """
        if not sharpOnly or not isinstance(sharpOnly,tuple):
            return
        self.posDict={trial:pos 
                      for trial,pos in self.posDict.items() 
                      if self.et[trial] >= sharpOnly[0] and self.et[trial] <=sharpOnly[1]}



#----------------------------------------------------------------------------------------------------------------------
if "__file__" not in dir():
    #------------------------------------------------
    session='Rat265_2018_07_13_14_39'
    data=Data(root,animal=session[:6],experiment=session,param=defaultParam,redoPreprocess=False,PrintWarning=True)
    plt.figure(figsize=(20,10))
    plt.subplot(221)
    a=plot_forward_running_speed(data)
    plt.subplot(222)
    plot_positions(data)

In [ ]:
if "__file__" not in dir():
    trial=35
    print(data.entranceTime[trial])
    plt.plot(data.timeTreadmill[trial],data.position[trial])
    plt.plot(data.timeTreadmill[trial][:data.stopFrame[trial]],data.position[trial][:data.stopFrame[trial]],'r')
    plt.axhline(data.position[trial][data.stopFrame[trial]-1])
    plt.axvline(data.entranceTime[trial])

In [ ]:
#----------------------------------------------------------------------
def get_positions_array_end_AllTrials(data,onlyGood=False,raw=False):
    '''
    Returns array of position, align on detected end, between minTime and zero (0=end)
    The dectect end can be None, if so the trial aligned to the treadmill stop time
    If one position is too short (detected end is before minTime), 
      the position is pad at the beginning with nan values
    '''
    if raw:
        posDict=data.rawPosition
    else:
        posDict=data.position
    #number of frames to keep 
    cs=data.cameraSamplingRate
    size=np.max(data.maxTrialDuration)*cs#int(abs(minTime*cs))
    #put all positions in a 2D array
    allTraj=[]
    for trial in posDict:
        if onlyGood and (trial not in data.goodTrials):
            continue
        #index where the steretyped trajectory ends
        endIndex=data.indexEndTrial[trial]
        if isNone(endIndex):
            #no end was detected, take stop treadmill time
            endIndex=data.stopFrame[trial]

        #Position is cut between (end -minTime) and end.
        #At minima, the cut should begin at startFrame
        startIndex=data.startFrame[trial]
        pos=posDict[trial][startIndex:endIndex]
       
        if len(pos)<(size):
            #pos=np.append([pos[0]] * (size-len(pos)),pos) #if too short, pad the start with a plateau (Nan would cause trouble to compute speed later)
            pos=np.append([np.nan] * (int(size)-len(pos)),pos)
        allTraj.append(pos)
    allTraj=np.asarray(allTraj)
    return allTraj
#---------------------------------------------------------------------------------------------------------------------
def plot_position_align_end_AllTrials(data,xUpLimit=5,onlyGood=False):
    '''
    xUpLimit is the x axis max limit
    onlyGood is whether to consider only good trials
    nb: data.timeEndTrial is computed with data.position
    '''
    cs=data.cameraSamplingRate
    for trial in data.position:
        end=data.timeEndTrial[trial]
        stop=data.stopFrame[trial]
        start=data.startFrame[trial]#+data.cameraToTreadmillDelay*data.cameraSamplingRate
        indexStart=np.argmax(data.position[trial])
        indexEnd=data.indexEndTrial[trial]
        if isNone(end):
            end=data.timeTreadmill[trial][data.stopFrame[trial]-1]
            indexEnd=data.stopFrame[trial]
        color="green"
        if trial not in data.goodTrials:
            if onlyGood:
                continue
            color="red"
        plt.plot(data.timeTreadmill[trial][start:stop]-end,data.position[trial][start:stop],color=color)
        plt.plot(data.timeTreadmill[trial][indexStart:indexEnd]-end,data.position[trial][indexStart:indexEnd],color="b",lw=2)
    plt.xlim([-15,xUpLimit])
    plt.ylabel("Position (cm/s)")
    plt.xlabel("Time relative to detected end (s)")
    plt.axvline(0,color="b",ls="--")
    plt.axvline(-np.mean(data.goalTime),color="k",ls="--")
    w="correct " if onlyGood else ""
    
    allTraj=get_positions_array_end_StereotypedTrials(data,onlyGood=onlyGood,raw=False)
    df = pd.DataFrame(allTraj.transpose())
    if allTraj.shape[0]<3:
        title="Not enough trials"
        med=np.nan
    else:
        corMatrix=np.array(df.corr())
        coeff=corMatrix[np.triu_indices(corMatrix.shape[0],1)]
        med=np.nanmedian(coeff)
    plt.title("Aligned to treadmill stop or minima of steretyped trajectory, \n median r=%.2f" %(med))    
    plt.axvspan(-np.max(data.maxTrialDuration),0,alpha=0.2,color="grey")
    return med

#----------------------------------------------------------------------------------------------------------------------
if "__file__" not in dir():

    plt.figure(figsize=(20,10))
    plt.subplot(221)
    plot_position_align_end_AllTrials(data,onlyGood=False)
    plt.subplot(222)
    plot_position_align_end_AllTrials(data,onlyGood=True)

### Plotting the lick data

In [ ]:
def plot_lick_raster(data,axes=None,maxDis=.5,plot_inset=True):
    Col=['g','r']
    if axes is None:
        ax=plt.gca()
    else:
        ax=axes
    lickTimes=[]
    for trial in data.trials:
        color=Col[0] if trial in data.goodTrials else Col[1]
        treadmillOffTime=data.timeTreadmill[trial][-1]-data.interTrialDuration[trial]+data.cameraToTreadmillDelay
        lickTimes.append(data.lickTime[trial]-treadmillOffTime-data.cameraToTreadmillDelay)
        ax.plot(lickTimes[-1],[trial]*len(data.lickTime[trial]),marker='|',markersize=4, markeredgecolor=None,linestyle = 'None',color=color)
        ax.axhspan(ymin=trial-.5,ymax= trial+.5,facecolor=color,edgecolor=None,alpha=.2)
    ax.vlines(0,0,data.nTrial+1,linewidth=.5)
       
    ax.set_xlim([-max(data.maxTrialDuration),max(data.interTrialDuration)+1])
    ax.set_ylim([0,data.nTrial+1])
    ax.set_xticks([-10,-data.goalTime[-1],0,max(data.interTrialDuration)])
    ax.set_title('Lick Raster')
    ax.set_xlabel('time relative to treadmill stop')
    ax.set_ylabel('trial #')
    
    if plot_inset:
        winLen=.5
        overlap=0.9
        goodRate=[]
        badRate=[]
        zero,end=ax.get_xlim()
        for trial in data.trials:
            Rate,Range=compute_rate(lickTimes[trial], winLen=winLen, overlap=overlap, zero=zero, end=end)
            if trial in data.goodTrials:
                goodRate.append(Rate)
            else:
                badRate.append(Rate)
        goodRate=np.array(goodRate)
        badRate=np.array(badRate)
        insetAx=inset.inset_axes(ax,width="100%", height="10%", loc=8)
        insetAx.plot(Range,np.mean(goodRate,axis=0),'g')
        insetAx.plot(Range,np.mean(badRate,axis=0),'r')
#         insetAx.fill_between(Range,np.mean(goodRate,axis=0)-np.std(goodRate,axis=0),np.mean(goodRate,axis=0)+np.std(goodRate,axis=0),'g',alpha=.25)
#         insetAx.fill_between(Range,np.mean(badRate,axis=0)-np.std(badRate,axis=0),np.mean(badRate,axis=0)+np.std(badRate,axis=0),'r',alpha=.25)
        
        insetAx.set_xticks([])
        insetAx.set_xlim([zero,end])
        insetAx.yaxis.set_ticks_position('right')
#         insetAx.set_title('lick rate (Hz)')
        insetAx.text(insetAx.get_xlim()[0],insetAx.get_ylim()[1],"win size={}\noverlap={}".format(winLen,overlap),ha='left',va='top',fontsize=10)
        insetAx.set_frame_on(False)
        ax.set_ylim([-.15*data.nTrial,data.nTrial+1])
        ax.set_yticks(range(0,data.nTrial,10))
        assert ax.get_xlim() == insetAx.get_xlim()
    
    return lickTimes

In [ ]:
if "__file__" not in dir():
    session="Rat227_2018_05_11_12_06"
    data=Data(root,animal=session[:6],experiment=session,param=defaultParam,redoPreprocess=False,PrintWarning=True)
#     i=38
#     out,Range=compute_rate (x=data.lickTime[i],winLen=1,overlap=0.95,zero=0,end=30)
#     plt.plot(data.lickTime[i],np.ones(len(data.lickTime[i])),'|')
#     plt.plot(Range,out)
#     inset.inset_axes(plt.gca(),".4","2",3)
#     plt.figure(figsize=(8,10))
#     a=plot_lick_raster(data)



    lickTrdStop=plot_lick_raster(data,plot_inset=False)
    plt.close()
    posLicks=[np.array(lick)[np.array(lick)>0] for i,lick in enumerate(lickTrdStop) if len(lick)>0 and i in data.goodTrials]
    firstLick=[i[0] for i in posLicks if len(i)>0]
    np.mean(firstLick)


### Speed

Treadmill speed has to taken into account to compute rat speed

` speed= treadmill speed - np.diff(position)*camera sampling rate `

positive speed= rat moving towards the front of the treadmill  
zero speed= rat not moving, dragged towards the end by the treadmill  
negative speed= rat moving towards the end, faster than the treadmill  

When the rat cross the beam, does the treadmill stop ? No it depends on its entrance time, the treadmill does not stop when the trial is bad (entrance time< goal time)

### Acceleration

` acceleration= np.diff(speed)*camera sampling rate `

In [ ]:
#---------------------------------------------------
def get_position_treadmillOn(data,trial):
    '''
    Compute position between treadmill start and treadmill stop, for one trial
    '''
    pos=data.position[trial]
    time=data.timeTreadmill[trial]
    start=data.startFrame[trial]   
    stoptime=time[-1]
    cs=data.cameraSamplingRate
    if data.entranceTime[trial]>=data.goalTime[trial]:
        stoptime = data.entranceTime[trial]
    stopList=np.where(time>=stoptime)[0]
    try:
        stop=stopList[0]
    except IndexError:
        stop=-1
    if data.dataType=="behav_param":
        stop=data.stopFrame[trial] 
    posCutted=pos[start:stop]
    timeCutted=time[start:stop]
    return posCutted,timeCutted
#---------------------------------------------------
def speed_from_pos(data,position,trial,sigmaSpeed=0.3):
    '''
    speed= (diff pos)* camera sampling rate -  treadmill speed 
    Speed can't be inferior to "- treadmill speed" (or even 0 ?)
    positive speed= rat moving towards the front of the treadmill
    negative speed= rat moving towards the end, faster than the treadmill
    '''
    cs=data.cameraSamplingRate
    tr=data.treadmillSpeed[trial]
    speed= tr - np.diff(position)*cs 
    speed[speed<(-tr)]=-tr
    speed=smooth(speed,sigmaSpeed*cs)
    return speed
#----------------------------------------------------------------------------------------------------------------------
def get_speed_treadmillON(data,trial,sigmaSpeed=0.3):
    '''
    Compute speed between treadmill start and treadmill stop, for one trial
    '''
    cs=data.cameraSamplingRate
    posCutted,timeCutted =  get_position_treadmillOn(data,trial)
    timeSpeed=(timeCutted[1:]+timeCutted[:-1])/2.0
    speed=speed_from_pos(data,posCutted,trial,sigmaSpeed)
    return speed,timeSpeed
#----------------------------------------------------------------------------------------------------------------------
def get_speed_end(data,trial,sigmaSpeed=0.3,minTime=-9):
    '''
    speed in the last 9 seconds (minTime=-9) (zero=detected end), for one trial
    '''
    cs=data.cameraSamplingRate
    endIndex=data.indexEndTrial[trial]
    if isNone(endIndex):
        return False, False
    pos=data.position[trial]
    cs=data.cameraSamplingRate
    startIndex=int(max(data.startFrame[trial],endIndex+(minTime*cs)))
    posCutted=pos[startIndex:endIndex]
    time=data.timeTreadmill[trial]-data.timeEndTrial[trial]
    timeCutted=time[startIndex:endIndex]
    timeSpeed=(timeCutted[1:]+timeCutted[:-1])/2.0
    speed=speed_from_pos(data,posCutted,trial,sigmaSpeed)
    return speed, timeSpeed
#----------------------------------------------------------------------------------------------------------------------
def plot_position_speed_acceleration(data,trial,sigmaSpeed=0.3):
    #plot position
    cs=data.cameraSamplingRate
    pos,time=get_position_treadmillOn(data,trial)
    plt.plot(time,pos,"k-",label="position")
    plt.ylabel("position (cm)")
    plt.xlabel("time (s)")
    # plot speed
   
    speed,timeSpeed=get_speed_treadmillON(data,trial,sigmaSpeed) 
    ax2=plt.gca().twinx()
    ax2.plot(timeSpeed,speed,color="red",label="speed")
    ax2.yaxis.label.set_color('red')
    ax2.tick_params(axis='y', colors='red') 
    ax2.set_ylabel("speed")
    plt.xlim([time[0],time[-1]])
    #plot acceleration
    timeAcceleration=timeSpeed[1:]
    cs=data.cameraSamplingRate
    acceleration= np.diff(speed)*cs
    ax2.plot(timeAcceleration,acceleration,color='blue',label="acceleration")
    
    #vertical and horizontal lines (treadmill speed, entrance time, startFrame)
    ax2.axhline(data.treadmillSpeed[trial],ls="--",color="g")
    ax2.axhline(0,ls="--",c="blue")
    ax2.axhline(-data.treadmillSpeed[trial],ls="--",color="g")
    plt.axvline(data.entranceTime[trial],color="purple")
    plt.axvline(data.maxTrialDuration[trial])
    plt.axvline(0,color="purple")

#----------------------------------------------------------------------------------------------------------------------
if "__file__" not in dir():
    plt.figure(figsize=(10,5))
    trial =5
    plt.subplot(211)
    plt.title("kinematics with raw time binning calculated here")
    plot_position_speed_acceleration(data,trial,sigmaSpeed=0.3)
    plt.xlim(0,25)
    plt.subplot(212)
    plt.title("Binned kinematics based on the class attributes")
    plt.plot(data.timeBin,data.positionBin[trial],"ko-",label="position")
    plt.plot(data.timeBin,data.speedSmoothBin[trial],"ro-",label="speed")
    plt.plot(data.timeBin,data.accelerationOnSpeedSmoothBin[trial],"bo-",label="acceleration")
    plt.legend(loc="best")
    plt.xlim(0,25)

In [ ]:
def plot_kinematic_histo(data,kinematic="speedSmoothBin",binSize=1,title="Distribution of speed",xlablel="speed(cm/s)",color='b',alpha=1,onlyGood=False):
    k=np.asarray([])
    for trial in data.trials:
        if (not onlyGood) or (trial in data.goodTrials):
            k=np.append(k,data.__dict__[kinematic][trial])          
    #check that the vector is not empty
    if len(k)==0:
        return np.nan   
    #histogram between min and max speed, with binSize
    k=k[~np.isnan(k)]
    mink=np.nanmin(k)
    maxk=np.nanmax(k)
    hist,bins=np.histogram(k,np.arange(mink,maxk+1,binSize),density=True)
    #plot histograme
    centers=(bins[:-1]+bins[1:])/2.0
    plt.bar(centers,hist,width=binSize,color=color,alpha=alpha)
    #title, labels
    if onlyGood:
        title+=" - %s good trials"%(len(data.goodTrials))
    plt.title(title)
    plt.xlabel(xlablel+", binSize=%s"%binSize)
    plt.ylabel("Density")
    
    #vertical lines for 0 and treadmill speed
    plt.axvline(0,color="r",ls="--")
    if kinematic=="speedSmoothBin":
        plt.axvline(np.nanmean(data.treadmillSpeed),color="c",ls="--")
        plt.xlim([-20,100])
    else:
        plt.xlim([-100,100])
    plt.ylim([0,0.15])
    return hist
#----------------------------------------------------------------------------------------------------------------------
if "__file__" not in dir():
    hist=plot_kinematic_histo(data,kinematic="speedSmoothBin",title="Speed ",xlablel="speed(cm/s)")
    plt.figure()
    hist=plot_kinematic_histo(data,kinematic="accelerationOnSpeedSmoothBin",title="Acceleration ",xlablel="acceleration(cm/s)")

In [ ]:
def setInterruptionBefore(data):
    rewardType = read_file(data, 'reward dispensing mode')
    rewardStart = read_file(data, 'if interruption before',valueType=float)
    return [rewardType[0], rewardStart[0]]


def plot_entrance_time(data,smoothSigma=2):
    #Get the entrance time for the correctly tracked trials
    entrance=[data.entranceTime[trial] for trial in data.trials]
    if len(entrance)<1:
        return
    #Colors for good/bad trials
    colors=["green" if t in data.goodTrials else "red" for t in data.trials]
    #Scatter plot entrance time/ trial
    plt.scatter(data.realTrials,entrance,color=colors,marker="o")
    #Smooth line entrane time/ trial
    smoothEntrance=smooth(entrance,smoothSigma)
    plt.plot(data.realTrials,smoothEntrance,"b--",linewidth=2)
    #Horizontal line: maxTrial duration
    plt.plot(data.realTrials,[data.maxTrialDuration[t] for t in data.trials],"k--")
    #mean goal time
    goal=np.nanmean(data.goalTime)
    plt.axhspan(0,goal,alpha=0.2,color="grey")
    sesionpercent=np.round(len(data.goodTrials)/float(data.nTrial) *100)
    entrancepercentile=np.nanpercentile(entrance,[25,75],axis=0)
    spread=np.round(entrancepercentile[1]-entrancepercentile[0],1)
    
    try:
        # Masoud: Add interruption before in case of KO/Ok
        interruptionBefore = setInterruptionBefore(data)
        if interruptionBefore[0] == "OK/KO":
            plt.axhline(y=interruptionBefore[1], xmin=0, xmax=1, color="black", linestyle="--", linewidth=1.5)
    except:pass
    
    #limits, labels, title
    plt.ylim([0,np.nanmax(data.maxTrialDuration)])
    plt.xlim([0,data.nTrial+1])
    plt.grid()
    plt.ylabel('Entrance Time (s)')
    plt.xlabel('Trial')
    plt.title("%s good trials, %s %% correct, spread: %s" %(len(data.goodTrials),sesionpercent,spread))

#----------------------------------------------------------------------------------------------------------------------
def plot_correlation_entrance_time(data,removeTimeout=False):    
    #entrance time with or without timeout
    if removeTimeout:
        entrance=[e for e,m in zip(data.entranceTime,data.maxTrialDuration) if e<m]
    else:
        entrance=data.entranceTime
    #remove nan/None
    entrance=[e for e in entrance if not isNone(e)]
    #check if not empty
    if not entrance:
        plt.title("No entrance times")
        return np.nan
    #plot entrance time n/entrance time n+1
    plt.plot(entrance[:-1],entrance[1:],"ko")
    plt.xlabel("Entrance time trial n")
    plt.ylabel("Entrance time trial n+1")
    #limits
    goal=np.nanmean(data.goalTime)
    m=max(goal,max(entrance))
    plt.ylim([0,m])
    plt.xlim([0,m])
    #diagonal and xspan
    plt.plot([0,m],[0,m],"k--")
    plt.axhspan(0,goal,alpha=0.2,color="grey")
    plt.axvspan(0,goal,alpha=0.2,color="grey")
    #spearman
    spearMan=scipy.stats.spearmanr(entrance[:-1],entrance[1:])
    if spearMan[1]<0.001:
        pvalue='p<0.001'
    else:
        pvalue='p= %.3f'%spearMan[1]
    plt.title("Correlation entrance times, Spearman r= %.2f, %s"%(spearMan[0],pvalue));  
    return spearMan[0]

#----------------------------------------------------------------------------------------------------------------------
def std_from_goaltime(data,removeTimeout=False):    
    #entrance time with or without timeout
    if removeTimeout:
        entrance=[e for e,m in zip(data.entranceTime,data.maxTrialDuration) if e<m]
    else:
        entrance=data.entranceTime
    #remove nan/None
    entrance=[e for e in entrance if not isNone(e)]
    #check if not empty
    if not entrance:
        return np.nan
    goal=np.nanmean(data.goalTime)
    #std from goal time
    ErrorStd=np.nanstd(np.abs(entrance-goal))
    #print(ErrorStd)

    return ErrorStd
    
#----------------------------------------------------------------------------------------------------------------------
if "__file__" not in dir():
    
    x=std_from_goaltime(data)
    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plot_entrance_time(data)
    plt.subplot(122)
    plot_correlation_entrance_time(data)
    #plt.xlim([0,25]),plt.ylim([0,25])

## Further Behavioral Analysis

In [ ]:
def entrance_time_succession(data, etRange=(6,8)):
    """
    This function returns number of successive trials 
    with ET within the given range
    """
    assert len(etRange)==2 and etRange[1]>etRange[0], "bad entrance time range"
    et=data.entranceTime
    goodEt=np.logical_and(et>=etRange[0],et<=etRange[1])
    #succesive Trues are the diff of the False values of the following array:
    goodEt=np.r_[False,goodEt,False]
    badTrials= np.where(goodEt==False)[0]
    successiveGood=np.diff(badTrials)-1
    
    return successiveGood

In [ ]:
def correct_succession_score(data, etRange=(6,8), minSuccession=3):
    successiveGood= entrance_time_succession(data, etRange)
    successiveGood=successiveGood[successiveGood>=minSuccession]
    successiveScore=np.sum(successiveGood-minSuccession+1)
    return successiveScore

In [ ]:
if "__file__" not in dir():
    session="Rat227_2018_03_16_15_21"
    data=Data(root,animal=session[:6],experiment=session,param=defaultParam,redoPreprocess=False,PrintWarning=True)
    a=correct_succession_score(data)
    print(a)

# Trials doing the motor sequnece

In [ ]:
def sequential_trials(data, xFront=30, xBackMin=50, Tolerance=.5):
    """
    Tolerance: in seconds, the tolerable duration of interruption ins moving backward, and forward
    """

    data.position_correction()
    detect_trial_end(data,trials=data.position.keys())
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)

    seqTrial=[]
    fireTime=[]
    for trial,pos in data.position.items():
        #starting from the front
        if pos[trialStart]>xFront and pos[0]>xFront:
            continue
        
        #not completing the trial
        if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
            continue

        trialPos=pos[trialStart:trialStart+int(cs*data.entranceTime[trial])+1]
        
        #moving sufficiently backward!
        if trialPos.max() < xBackMin:
            continue
        
        #moving backward uninterrupted
        backwardEpoch=trialPos[:trialPos.argmax()]
        if sum(np.diff(backwardEpoch)<0) > Tolerance*cs:
            continue

        #moving forward uninterrupted
        forwardEpoch=trialPos[trialPos.argmax():]
        if sum(np.diff(forwardEpoch)>0) > Tolerance*cs:
            continue
        
        #The trial IS indeed sequential
        seqTrial.append(trial)
        fireTime.append((trialPos.argmax()/cs))

    return np.array(seqTrial),np.array(fireTime)

In [ ]:
if "__file__" not in dir():
    session='Rat218_2018_04_09_09_35'
    data=Data(root,session[:6],session,param={},redoPreprocess=False, saveAsPickle=False);
    seqTrials,fireTimes=sequential_trials(data)
    score=len(seqTrials)/data.nTrial
    print(score)

### Waiting Time (how long the animal waits before moving)

In [ ]:
def animal_velocity(data,trial,xFront=30,sigmaSpeed=0.3):
    '''
    speed= (diff pos)* camera sampling rate -  treadmill speed 
    Speed can't be inferior to "- treadmill speed" (or even 0 ?)
    positive speed= rat moving towards the front of the treadmill
    negative speed= rat moving towards the end, faster than the treadmill
    '''
    cs=data.cameraSamplingRate
    tr=data.treadmillSpeed[trial]
    posCutted,_ =  get_position_treadmillOn(data,trial)
    speed= tr - np.diff(posCutted)*cs 
    speed=smooth(speed,sigmaSpeed*cs)
    return speed

def waiting_time(data, xFront=30):
    #Very sensitive parameter, do not play with lightly!
    SPEED_TOL=10
    
    detect_trial_end(data,trials=data.position.keys())
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)
    
    waitTrial=[]
    waitTime=[]
    for trial,pos in data.position.items():
#         if trial==16:breakpoint()
        #starting from the front
        if pos[trialStart]>xFront:
            continue
        #getting the instantaneous speed
        speed=animal_velocity(data,trial)
        
        idx=np.where(speed>abs(SPEED_TOL))[0]
        if len(idx) > 0: #first time speed is faster than speedTolerance
            waitTime.append(idx[0]/cs)
            waitTrial.append(trial)
        
        else: #no waiting, hence, waiting time = 0
            waitTime.append(0)
            waitTrial.append(trial)
            
    return np.array(waitTime),np.array(waitTrial)

In [ ]:
if "__file__" not in dir():
    session='Rat218_2018_04_09_09_35'
    data=Data(root,session[:6],session,param={},redoPreprocess=False, saveAsPickle=False);
    waitTimes,waitTrial=waiting_time(data)

    score=np.mean(waitTimes)
    print(score)

In [ ]:
class sequentialTrials:
    def __init__(self, data, xFront=30, xBackMin=50, Tolerance=.5):
        data.position_correction()
        detect_trial_end(data,trials=data.position.keys())
        self.data=data
        self.xFront=xFront
        self.xBackMin=xBackMin
        self.Tolerance=Tolerance
        self.cs=int(data.cameraSamplingRate)
        self.trialStart=int(self.cs*data.cameraToTreadmillDelay)
        
        self.seq_score=None
        self.seqTrials=None
          
    def SeqScore(self):
        if self.seq_score is None:
            seqTrials,_=self._compute_sequential_trials()
            score=len(seqTrials)/self.data.nTrial
            self.seqTrials=seqTrials
            self.seq_score=score
        return score
    
    def WaitingTime(self, TH=.35):
        if self.seqTrials is None:
            self.SeqScore()
        
        if self.seq_score < TH:
            waitTimes,_=self._compute_waiting_time2()
        else:
            waitTimes,_=self._compute_waiting_time ()
        
        self.waiting= np.nanmean (waitTimes)
        
        return self.waiting
    
    def MaxPosition(self, th=0.4):
        if self.SeqScore() < th:
            return np.nan
        else:
            maxPos,_=self._compute_max_pos()
            return maxPos
        
    def _compute_waiting_time(self):
        """
        computes the immobility duration in each sequential trial,
        """
        #Very sensitive parameter, do not play with lightly!
        SPEED_TOL=10
        
        xFront=self.xFront
        cs=self.cs
        trialStart=self.trialStart
        data=self.data

        waitTrial=[]
        waitTime=[]
        for trial,pos in data.position.items():
            if trial not in self.seqTrials:
                continue
            
            #getting the instantaneous speed
            speed=self._animal_velocity(trial)

            idx=np.where(speed>abs(SPEED_TOL))[0]
            if len(idx) > 0: #first time speed is faster than speedTolerance
                waitTime.append(idx[0]/cs)
                waitTrial.append(trial)

            else:
                continue
        
        return np.array(waitTime),np.array(waitTrial)
    
    def _animal_velocity(self,trial,sigmaSpeed=0.3):
        '''
        speed= (diff pos)* camera sampling rate -  treadmill speed 
        Speed can't be inferior to "- treadmill speed" (or even 0 ?)
        positive speed= rat moving towards the front of the treadmill
        negative speed= rat moving towards the end, faster than the treadmill
        '''
        cs=self.cs
        tr=self.data.treadmillSpeed[trial]
        posCut,_ =  get_position_treadmillOn(self.data,trial)
        speed= tr - np.diff(posCut)*cs 
        speed=smooth(speed,sigmaSpeed*cs)
        return speed
    
    def _compute_waiting_time2(self,minDisplacement=30):
        """
        applies the immobility rule to the median average trajesctory of session
        """
        SPEED_TOL=10
        #based on the average session trajectory
        cs=self.cs
        tr=self.data.treadmillSpeed[self.data.nTrial-5]
        
        allTraj=get_positions_array_beginning(self.data,onlyGood=False,raw=False)
        
        #Check that the array is not empty
        if allTraj.shape[0]==0:
            return np.array([np.nan]),[]

        #keeping data where 70% of points exist
        nanSum=np.sum(np.isnan(allTraj),axis=0)
        try:
            maxTraj=np.where(nanSum>.3*allTraj.shape[0])[0][0]
        except IndexError:
            maxTraj=allTraj.shape[0]
        
        medTraj=np.nanmedian(allTraj,axis=0)[:maxTraj]
        
        if medTraj.max() - medTraj[0] < minDisplacement:
            return np.array([np.nan]), medTraj
        
        speed= tr - np.diff(medTraj)*cs 
        speed=smooth(speed,.3*cs)
        
        idx=np.where(speed>abs(SPEED_TOL))[0]
        if len(idx) > 0: #first time speed is faster than speedTolerance
            waitTime=[idx[0]/cs]
        else:
            return np.array([np.nan])
        
        return np.array(waitTime),medTraj

    def _compute_max_pos(self):
        #based on the average session trajectory
        cs=self.cs
        tr=self.data.treadmillSpeed[self.data.nTrial-5]
        
        allTraj=get_positions_array_beginning(self.data,onlyGood=False,raw=False)
        
        #Check that the array is not empty
        if allTraj.shape[0]==0:
            return np.array([np.nan]),[]

        #keeping data where 70% of points exist
        nanSum=np.sum(np.isnan(allTraj),axis=0)
        try:
            maxTraj=np.where(nanSum>.3*allTraj.shape[0])[0][0]
        except IndexError:
            maxTraj=allTraj.shape[0]
        
        medTraj=np.nanmedian(allTraj,axis=0)[:maxTraj]
        
        maxPos=np.amax(medTraj)
        return np.array(maxPos),medTraj
    
    def _compute_sequential_trials(self):
        """
        Tolerance: in seconds, the tolerable duration of interruption ins moving backward, and forward
        """
        data=self.data
        trialStart=self.trialStart
        cs=self.cs
        xFront=self.xFront
        xBackMin=self.xBackMin
        Tolerance=self.Tolerance
        
        seqTrial=[]
        fireTime=[]
        for trial,pos in data.position.items():
            #starting from the front
            if pos[trialStart]>xFront and pos[0]>xFront:
                continue

            #not completing the trial
            if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
                continue

            trialPos=pos[trialStart:trialStart+int(cs*data.entranceTime[trial])+1]

            #moving sufficiently backward!
            if trialPos.max() < xBackMin:
                continue

#             #moving backward uninterrupted
#             backwardEpoch=trialPos[:trialPos.argmax()]
#             if sum(np.diff(backwardEpoch)<0) > Tolerance*cs:
#                 continue

#             #moving forward uninterrupted
#             forwardEpoch=trialPos[trialPos.argmax():]
#             if sum(np.diff(forwardEpoch)>0) > Tolerance*cs:
#                 continue

            #The trial IS indeed sequential
            seqTrial.append(trial)
            fireTime.append((trialPos.argmax()/cs))

        return np.array(seqTrial),np.array(fireTime)

In [ ]:
if "__file__" not in dir():
    session='Rat299_2018_12_19_14_22'
    data=Data(root,session[:6],session,param={},redoPreprocess=False, saveAsPickle=False);
    a=sequentialTrials(data)
    score=a.SeqScore()
    waitTime=a.WaitingTime()
    a,_=a._compute_max_pos()
    print(f'seq=\t{score}\n\nWaitTime=\t{waitTime}\n\nWaitTime3=\t{a}')